## Examples on using pydggapi endpoints

This notebook illustrates some sample uses of the OGC DGGS-API endpoint. The API endpoints can be categorised into three categories.

**DGGS API endpoints:**

The queries below work on all datasets defined in the `dggs_api_config.json` file. The API only returns results if the zones of the query exist in any of the collections. The API endpoints are organised by conformance class.

- Core conformance class
  - (`/`) : landing page
  - (`/dggs`) : query supported dggrs 
  - (`/dggs/<dggrs_id>`) : query dggrs definition 
  - (`/dggs/<dggrs_id>/zones/<zone_id>`) : query zone info 
- Zone Query conformance class
  - (`/dggs/<dggrs_id>/zones`) : listing zones of a specific bounding box.
- Zone Data Retrieval conformance class 
  - (`/dggs/<dggrs_id>/zones/<zone_id>/data`) : get data for a specific zone 

**Collection DGGRS API endpoints:**

Apart from query zones for all datasets, the above queries can be used for a specific collection:
- collections

**Tiles API for collection:**

- tiles-json
- tiles

### Setup
All api endpoints work with HTTP GET requests. In this example, we primarily use the requests library to perform HTTP queries. 

The API returns JSON by default; it also supports returning various data types, which we will demonstrate in the upcoming section.

In [20]:
import requests
# to display the return json
import json
from IPython.display import display, JSON

# define the api root url
api_root="http://127.0.0.1:8000/dggs-api/v1-pre"

### DGGS API Core conformance class

#### landing page (`<api_root>/`)

The landing page provides links to the API definition.

In [34]:
# send the query, and get the return from the body text
query_return = requests.get(api_root).text
# load the return into a json object
query_return = json.loads(query_return)
# display it nicely with IPython display module
display(JSON(query_return))

<IPython.core.display.JSON object>

#### Query supported dggrs (`<api_root>/dggs`)

This query lists all DGGRs supported by the API instance in the list `dggrs`.

In [29]:
query_return = requests.get(api_root+'/dggs').text
query_return = json.loads(query_return)
display(JSON(query_return))

<IPython.core.display.JSON object>

#### Query dggrs definition (`<api_root>/dggs/<dggrs_id>`)
This query returns the definition and details of the dggrs

In [30]:
query_return = requests.get(api_root+'/dggs/igeo7').text
query_return = json.loads(query_return)
display(JSON(query_return))

<IPython.core.display.JSON object>

#### Query zone info (`<api_root>/dggs/<dggrs_id>/zones/<zone_id>`)
This query returns the zone's properties (zone's centroid, zone's geometry and area etc.)

In [32]:
query_return = requests.get(api_root+'/dggs/igeo7/zones/0001022010').text
query_return = json.loads(query_return)
display(JSON(query_return))

<IPython.core.display.JSON object>

### DGGS API Zone Query conformance class

#### Zone Query (`<api_root>/dggs/<dggrs_id>/zones`)

This query accepts the following query parameters: 

- bbox
  - Bounding box of the query. The bounding box is provided as four coordinates, in the order of minx,miny,maxx and maxy
- zone_level
  - For specifying a level at which to return a list of DGGRS zones using a zone_level query parameter.
- compact_zone
  - For specifying whether to retrieve a list of DGGRS zones using a compact_zones query parameter.
- parent_zone
  - For specifying a parent zone within which to restrict zone listing using a parent_zone query parameter.
- geometry
  - Specify the return geometry (`zone-centroid` or `zone-region`) 
- filter
  - A CQL2 filter string
- datetime
  - Specify the datetime interval

Users can specify the return type in the HTTP header or using the `format query string`. The zone query supports the following return types: 
- `?f=json` or set `application/json` in the http header
- `?f=geojson` or `application/geo+json` in the http header


In [ ]:

query_return = requests.get(api_root+'/dggs/igeo7/zones/0001022010').text
query_return = json.loads(query_return)
display(JSON(query_return))